In [406]:
'''
import the necessary packages
'''
#the module 'sys' allows istalling module from inside Jupyter
import sys

!{sys.executable} -m pip install numpy
import numpy as np

!{sys.executable} -m pip install pandas
import pandas as pd

#Natrual Language ToolKit (NLTK)
!{sys.executable} -m pip install nltk
import nltk

!{sys.executable} -m pip install sklearn
from sklearn import metrics
#from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import  CountVectorizer #bag-of-words vectorizer 
from sklearn.decomposition import LatentDirichletAllocation #package for LDA

# Plotting tools

from pprint import pprint
!{sys.executable} -m pip install pyLDAvis #visualizing LDA
import pyLDAvis
import pyLDAvis.sklearn

import matplotlib.pyplot as plt
%matplotlib inline

#ignore warnings about future changes in functions as they take too much space
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

#define text normalization function
%run ./Text_Normalization_Function.ipynb #defining text normalization function

Processing /Users/RickyGui/Library/Caches/pip/wheels/4f/85/2a/67a30aa6cf144eca0c159f337ce5166df2213c4cde9e699cbe/html_parser-0.2-py3-none-any.whl


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/RickyGui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/RickyGui/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/RickyGui/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/RickyGui/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

In [408]:
import pandas as pd

In [409]:
'''
Read the data
'''
review_data = pd.read_csv("./Amazon_Unlocked_Mobile.csv")

review_data = pd.DataFrame(review_data)

In [410]:
'''
Price Range and pre processing
'''
low_price_data = review_data[review_data["Price"]<=300]
mid_price_data = review_data[(review_data["Price"]>300) & (review_data["Price"]<=600)]
high_price_data = review_data[review_data["Price"]>600]
low_price_data = low_price_data[low_price_data["Reviews"].apply(lambda x: isinstance(x, str))]
mid_price_data = mid_price_data[mid_price_data["Reviews"].apply(lambda x: isinstance(x, str))]
high_price_data = high_price_data[high_price_data["Reviews"].apply(lambda x: isinstance(x, str))]

In [411]:
low_price_data = low_price_data.sample(frac = 0.073,random_state=10)
# random sampling data from the original set to keep the sample size balance with other sets
mid_price_data = mid_price_data.sample(frac = 0.38,random_state=10)
# random sampling data from the original set to keep the sample size balance with other sets

In [412]:
'''
Label the data based on ratings
Drop the rows with rating of 3
Rating >3  ------> positive
Rating <3  ------> negative
'''
low_price_data = low_price_data[low_price_data["Rating"] !=3]
mid_price_data = mid_price_data[mid_price_data["Rating"] !=3]
high_price_data = high_price_data[high_price_data["Rating"] !=3]
low_price_data["polarity"] = np.where(low_price_data["Rating"] >3, "positive", "negative")
mid_price_data["polarity"] = np.where(mid_price_data["Rating"] >3, "positive", "negative")
high_price_data["polarity"] = np.where(high_price_data["Rating"] >3, "positive", "negative")

In [413]:
low_price_review = low_price_data["Reviews"]
mid_price_review = mid_price_data["Reviews"]
high_price_review = high_price_data["Reviews"]
low_price_polarity = low_price_data["polarity"]
mid_price_polarity = mid_price_data["polarity"]
high_price_polarity = high_price_data["polarity"]

In [414]:
'''
Data cleaning
Move the string reviews to new list
'''
samsung_data_review = review_data[review_data["Brand Name"] == "Samsung"]
samsung_data_review.reset_index(drop=False, inplace=True)
samsung_cleaned = []
for ele in samsung_data_review["Reviews"]:
    if type(ele) == str:
        samsung_cleaned.append(ele)
#samsung_cleaned


In [415]:
apple_data_review = review_data[review_data["Brand Name"] == "Apple"]
apple_data_review.reset_index(drop=False, inplace=True)
apple_cleaned = []
for ele in apple_data_review["Reviews"]:
    if type(ele) == str:
        apple_cleaned.append(ele)
#apple_data_review

In [416]:
huawei_data_review = review_data[review_data["Brand Name"] == "Huawei"]
huawei_data_review.reset_index(drop=False, inplace=True)
huawei_cleaned = []
for ele in huawei_data_review["Reviews"]:
    if type(ele) == str:
        huawei_cleaned.append(ele)
#huawei_data_review

In [417]:
'''
=============================
Sumsung Normalize & Vectorize
=============================
'''
#normalize data
normalized_samsung_data_review = normalize_corpus(samsung_cleaned)
#normalized_samsung_data_review

#define a Bag-of-Words vecgtorizer
bow_vectorizer_news = CountVectorizer(max_features=1000)

#vectorize data
samsung_corpus = bow_vectorizer_news.fit_transform(normalized_samsung_data_review)

lda_samsung_reivew = LatentDirichletAllocation(n_components=10, max_iter=100,
                                     doc_topic_prior = 0.25,
                                     topic_word_prior = 0.25).fit(samsung_corpus)



In [418]:
'''
=====================
Sumsung visualization
=====================
'''
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()
#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.sklearn.prepare(lda_samsung_reivew, samsung_corpus, bow_vectorizer_news, mds='tsne')
#brand_review_data = review_data["Brand Name","Reviews"]
#brand_review_data

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
8      32.420952   8.115230       1        1  22.069588
2       0.648955  71.707100       2        1  17.092969
5      71.019684   9.971006       3        1  12.683513
9       3.785981  33.185135       4        1   8.659640
7     -42.666500  -9.222070       5        1   8.486959
1      -3.621689 -43.526516       6        1   7.634788
4      -3.632402  -4.171708       7        1   7.057496
0     -36.584400  35.804413       8        1   6.600471
6      44.391380  50.615383       9        1   5.277121
3      40.235809 -31.591665      10        1   4.437455, topic_info=        Term          Freq         Total Category  logprob  loglift
372     good  15228.000000  15228.000000  Default  30.0000  30.0000
377    great  14821.000000  14821.000000  Default  29.0000  29.0000
623    phone  72397.000000  72397.000000  Default  28.0000  28.0000
500     love   9948.000000   9948.000000  Default  27.0000  27.0000
756  samsung   9586.000000   9586.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
682  quality    603.768009   2574.003714  Topic10  -4.2062   1.6651
773   seller    716.816084   3740.858059  Topic10  -4.0345   1.4628
778  service    341.510890   2990.699971  Topic10  -4.7760   0.9452
137     cell    187.986486   2234.061619  Topic10  -5.3730   0.6399
893     time    181.254399   6781.451565  Topic10  -5.4095  -0.5070

[531 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         8  0.693452  16gb
0         9  0.295186  16gb
1         3  0.988657    2g
3         3  0.970774    3g
3        10  0.026597    3g
...     ...       ...   ...
991       7  0.430724   wow
991       8  0.166136   wow
994       1  0.198236  year
994       2  0.475493  year
994       6  0.325217  year

[897 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 3, 6, 10, 8, 2, 5, 1, 7, 4])

In [419]:
'''
=============================
Apple Normalize & Vectorize
=============================
'''
#normalize data
normalized_apple_data_review = normalize_corpus(apple_cleaned)
#normalized_samsung_data_review

#define a Bag-of-Words vecgtorizer
bow_vectorizer_news = CountVectorizer(max_features=1000)

#vectorize data
apple_corpus = bow_vectorizer_news.fit_transform(normalized_apple_data_review)

lda_apple_reivew = LatentDirichletAllocation(n_components=10, max_iter=100,
                                     doc_topic_prior = 0.25,
                                     topic_word_prior = 0.25).fit(apple_corpus)



In [420]:
'''
=====================
Apple visualization
=====================
'''
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()
#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.sklearn.prepare(lda_apple_reivew, apple_corpus, bow_vectorizer_news, mds='tsne')
#brand_review_data = review_data["Brand Name","Reviews"]
#brand_review_data

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
9      32.420952   8.115230       1        1  17.362825
6       0.648955  71.707100       2        1  16.172755
8      71.019684   9.971006       3        1  13.025083
4       3.785981  33.185135       4        1  10.555199
5     -42.666500  -9.222070       5        1   9.294537
1      -3.621689 -43.526516       6        1   8.618883
3      -3.632402  -4.171708       7        1   7.148025
2     -36.584400  35.804413       8        1   6.603533
7      44.391380  50.615383       9        1   5.796773
0      40.235809 -31.591665      10        1   5.422387, topic_info=           Term          Freq         Total Category  logprob  loglift
986        work  15933.000000  15933.000000  Default  30.0000  30.0000
381       great  13773.000000  13773.000000  Default  29.0000  29.0000
376        good  11242.000000  11242.000000  Default  28.0000  28.0000
508        love   7379.000000   7379.000000  Default  27.0000  27.0000
629       phone  40952.000000  40952.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
556        much    324.824521   2294.868415  Topic10  -4.5877   0.9595
780     service    260.634146   1906.398328  Topic10  -4.8079   0.9248
47      amazing    151.533266    582.128867  Topic10  -5.3502   1.5687
237     deliver    156.831237    687.720360  Topic10  -5.3159   1.4364
309  experience    155.224342    741.012697  Topic10  -5.3261   1.3515

[554 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
4         2  0.980307    3d
7         5  0.983895   3rd
9         2  0.997257    4s
11        2  0.467071    5s
11        5  0.110332    5s
...     ...       ...   ...
996       5  0.800802  year
997       2  0.338518   yes
997       5  0.124266   yes
997       8  0.359943   yes
997       9  0.173544   yes

[905 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 7, 9, 5, 6, 2, 4, 3, 8, 1])

In [421]:
'''
=============================
Huawei Normalize & Vectorize
=============================
'''
#normalize data
normalized_huawei_data_review = normalize_corpus(huawei_cleaned)
#normalized_samsung_data_review

#define a Bag-of-Words vecgtorizer
bow_vectorizer_news = CountVectorizer(max_features=1000)

#vectorize data
huawei_corpus = bow_vectorizer_news.fit_transform(normalized_huawei_data_review)

lda_huawei_reivew = LatentDirichletAllocation(n_components=10, max_iter=100,
                                     doc_topic_prior = 0.25,
                                     topic_word_prior = 0.25).fit(huawei_corpus)

In [422]:
'''
=====================
Huawei visualization
=====================
'''
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()
#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.sklearn.prepare(lda_huawei_reivew, huawei_corpus, bow_vectorizer_news, mds='tsne')
#brand_review_data = review_data["Brand Name","Reviews"]
#brand_review_data

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1  17.332162
5       0.648955  71.707100       2        1  16.479809
4      71.019684   9.971006       3        1  10.548791
2       3.785981  33.185135       4        1  10.167200
6     -42.666500  -9.222070       5        1   9.179245
7      -3.621689 -43.526516       6        1   8.855097
8      -3.632402  -4.171708       7        1   8.315457
3     -36.584400  35.804413       8        1   7.853252
9      44.391380  50.615383       9        1   5.993070
1      40.235809 -31.591665      10        1   5.275916, topic_info=        Term         Freq        Total Category  logprob  loglift
377    great   987.000000   987.000000  Default  30.0000  30.0000
137     card   375.000000   375.000000  Default  29.0000  29.0000
84   battery  1069.000000  1069.000000  Default  28.0000  28.0000
370     good  1289.000000  1289.000000  Default  27.0000  27.0000
805      sim   368.000000   368.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
370     good    43.227473  1289.778682  Topic10  -4.6044  -0.4537
842  speaker    29.215725   140.141359  Topic10  -4.9961   1.3741
776   screen    33.530051   972.012776  Topic10  -4.8584  -0.4249
132   camera    30.667108   571.146428  Topic10  -4.9477   0.0176
419   huawei    28.570046   778.548615  Topic10  -5.0185  -0.3631

[621 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
2         8  0.189496    2g
2         9  0.631652    2g
2        10  0.126330    2g
4         8  0.224466    3g
4         9  0.758907    3g
...     ...       ...   ...
996       1  0.361012  year
996       2  0.115323  year
996       3  0.300843  year
996       6  0.075211  year
996       7  0.140393  year

[1558 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 5, 3, 7, 8, 9, 4, 10, 2])

In [423]:
'''
=============================
Low price Normalize & Vectorize
=============================
'''
#normalize data
normalized_low_data_review = normalize_corpus(low_price_review)
#normalized_samsung_data_review

#define a Bag-of-Words vecgtorizer
bow_vectorizer_news = CountVectorizer(max_features=1000)

#vectorize data
low_corpus = bow_vectorizer_news.fit_transform(normalized_low_data_review)

lda_low_reivew = LatentDirichletAllocation(n_components=10, max_iter=100,
                                     doc_topic_prior = 0.25,
                                     topic_word_prior = 0.25).fit(low_corpus)



In [424]:
'''
=============================================
Low price range topic modeling visualization
=============================================
'''
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()
#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.sklearn.prepare(lda_low_reivew, low_corpus, bow_vectorizer_news, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
3      32.420952   8.115230       1        1  18.224572
4       0.648955  71.707100       2        1  17.767849
1      71.019684   9.971006       3        1  16.555855
8       3.785981  33.185135       4        1  12.173453
6     -42.666500  -9.222070       5        1   8.228504
7      -3.621689 -43.526516       6        1   6.922752
0      -3.632402  -4.171708       7        1   6.358610
2     -36.584400  35.804413       8        1   5.777390
5      44.391380  50.615383       9        1   5.293171
9      40.235809 -31.591665      10        1   2.697844, topic_info=       Term          Freq         Total Category  logprob  loglift
380   great   5546.000000   5546.000000  Default  30.0000  30.0000
375    good   5894.000000   5894.000000  Default  29.0000  29.0000
987    work   6364.000000   6364.000000  Default  28.0000  28.0000
510    love   3021.000000   3021.000000  Default  27.0000  27.0000
637   phone  24471.000000  24471.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
920     una     86.722262     88.948916  Topic10  -4.6285   3.5874
283  equipo     84.064713     86.290895  Topic10  -4.6596   3.5866
179     con    248.927261    341.002348  Topic10  -3.5740   3.2980
974      wi    116.193866    191.157619  Topic10  -4.3359   3.1149
328      fi    116.940841    196.337327  Topic10  -4.3295   3.0946

[523 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1         4  0.977992      2g
6         6  0.982374      4s
7         6  0.967585      5c
8         6  0.985658      5s
12        1  0.411833    able
...     ...       ...     ...
995       2  0.971181  xperia
996       1  0.031129    year
996       2  0.503248    year
996       3  0.376139    year
996       7  0.086901    year

[962 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 2, 9, 7, 8, 1, 3, 6, 10])

In [425]:
'''
=============================
Mid price Normalize & Vectorize
=============================
'''
#normalize data
normalized_mid_data_review = normalize_corpus(mid_price_review)
#normalized_samsung_data_review

#define a Bag-of-Words vecgtorizer
bow_vectorizer_news = CountVectorizer(max_features=1000)

#vectorize data
mid_corpus = bow_vectorizer_news.fit_transform(normalized_mid_data_review)

lda_mid_reivew = LatentDirichletAllocation(n_components=10, max_iter=100,
                                     doc_topic_prior = 0.25,
                                     topic_word_prior = 0.25).fit(mid_corpus)



In [426]:
'''
=============================================
Mid price range topic modeling visualization
=============================================
'''
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()
#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.sklearn.prepare(lda_mid_reivew, mid_corpus, bow_vectorizer_news, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
7      32.420952   8.115230       1        1  23.094065
3       0.648955  71.707100       2        1  17.240242
1      71.019684   9.971006       3        1  13.874498
9       3.785981  33.185135       4        1  10.438960
0     -42.666500  -9.222070       5        1   7.627367
4      -3.621689 -43.526516       6        1   7.109695
5      -3.632402  -4.171708       7        1   6.272239
8     -36.584400  35.804413       8        1   4.824265
2      44.391380  50.615383       9        1   4.818834
6      40.235809 -31.591665      10        1   4.699834, topic_info=         Term          Freq         Total Category  logprob  loglift
382     great   6119.000000   6119.000000  Default  30.0000  30.0000
632     phone  24246.000000  24246.000000  Default  29.0000  29.0000
377      good   5243.000000   5243.000000  Default  28.0000  28.0000
505      love   3975.000000   3975.000000  Default  27.0000  27.0000
982      work   6116.000000   6116.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
121       buy    241.998711   3389.636190  Topic10  -4.2038   0.4181
501      look    123.385179   1937.423909  Topic10  -4.8774   0.3038
213  customer     44.981279    379.961581  Topic10  -5.8865   0.9238
892      time     58.666877   2370.979234  Topic10  -5.6208  -0.6415
148     cheap     27.761987    332.555259  Topic10  -6.3690   0.5745

[531 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
1         1  0.341816   2nd
1         2  0.264131   2nd
1         7  0.186445   2nd
1        10  0.186445   2nd
3         4  0.987165    3g
...     ...       ...   ...
992       1  0.518475   yes
992       7  0.476718   yes
996       2  0.982869    z3
997       2  0.984937    z5
998       2  0.971296  zoom

[1005 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 4, 2, 10, 1, 5, 6, 9, 3, 7])

In [427]:
'''
=============================
high price Normalize & Vectorize
=============================
'''
#normalize data
normalized_high_data_review = normalize_corpus(high_price_review)
#normalized_samsung_data_review

#define a Bag-of-Words vecgtorizer
bow_vectorizer_news = CountVectorizer(max_features=1000)

#vectorize data
high_corpus = bow_vectorizer_news.fit_transform(normalized_high_data_review)

lda_high_reivew = LatentDirichletAllocation(n_components=10, max_iter=100,
                                     doc_topic_prior = 0.25,
                                     topic_word_prior = 0.25).fit(high_corpus)



In [428]:
'''
=============================================
High price range topic modeling visualization
=============================================
'''
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()
#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.sklearn.prepare(lda_high_reivew, high_corpus, bow_vectorizer_news, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      32.420952   8.115230       1        1  16.499799
4       0.648955  71.707100       2        1  14.364818
7      71.019684   9.971006       3        1  12.669560
3       3.785981  33.185135       4        1  12.326477
1     -42.666500  -9.222070       5        1  10.712385
5      -3.621689 -43.526516       6        1   9.469672
6      -3.632402  -4.171708       7        1   6.846175
0     -36.584400  35.804413       8        1   6.208833
8      44.391380  50.615383       9        1   5.665294
9      40.235809 -31.591665      10        1   5.236987, topic_info=          Term         Freq        Total Category  logprob  loglift
374       good  5231.000000  5231.000000  Default  30.0000  30.0000
380      great  4931.000000  4931.000000  Default  29.0000  29.0000
505       love  3439.000000  3439.000000  Default  28.0000  28.0000
984       work  4318.000000  4318.000000  Default  27.0000  27.0000
452     iphone  3133.000000  3133.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
488       like   257.001169  2625.579997  Topic10  -4.0668   0.6254
558       much   140.319667  1246.122082  Topic10  -4.6719   0.7656
501       look   109.919988  1500.431331  Topic10  -4.9161   0.3357
664     pretty    58.577276   448.046147  Topic10  -5.5455   0.9149
148  cellphone    47.981947   311.895050  Topic10  -5.7450   1.0776

[570 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
5         5  0.237920    3g
5         6  0.748578    3g
6         5  0.973990   3gb
8         1  0.291858    4g
8         6  0.704132    4g
...     ...       ...   ...
995       1  0.221434  year
995       3  0.031972  year
995       4  0.169332  year
995       5  0.575491  year
999       3  0.965099    z3

[1082 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 8, 4, 2, 6, 7, 1, 9, 10])

In [431]:
from sklearn import metrics

In [432]:
'''
Data cleaning
Keep only the string reviews
'''
samsung_data_review = samsung_data_review[samsung_data_review["Reviews"].apply(lambda x: isinstance(x, str))]
apple_data_review = apple_data_review[apple_data_review["Reviews"].apply(lambda x: isinstance(x, str))]
huawei_data_review = huawei_data_review[huawei_data_review["Reviews"].apply(lambda x: isinstance(x, str))]

In [433]:
'''
==============================================
Supervised Machine Learning Sentiment Analysis
==============================================
'''
# Sumsung Data pre-processing
samsung_data_review.dropna(inplace=True)
samsung_data_review = samsung_data_review[samsung_data_review["Rating"] !=3]
samsung_data_review["polarity"] = np.where(samsung_data_review["Rating"] >3, "positive", "negative")
samsung_data_review_train = samsung_data_review.loc[:int(0.2*len(samsung_data_review["Reviews"]))]
samsung_data_review_test = samsung_data_review.loc[int(0.2*len(samsung_data_review["Reviews"])):]

In [434]:
# Sumsung Data pre-processing
apple_data_review.dropna(inplace=True)
apple_data_review = apple_data_review[apple_data_review["Rating"] !=3]
apple_data_review["polarity"] = np.where(apple_data_review["Rating"] >3, "positive", "negative")
apple_data_review_train = apple_data_review.loc[:int(0.2*len(apple_data_review["Reviews"]))]
apple_data_review_test = apple_data_review.loc[int(0.2*len(apple_data_review["Reviews"])):]

In [435]:
# Huawei Data pre-processing
huawei_data_review.dropna(inplace=True)
huawei_data_review = huawei_data_review[huawei_data_review["Rating"] !=3]
huawei_data_review["polarity"] = np.where(huawei_data_review["Rating"] >3, "positive", "negative")
huawei_data_review_train = huawei_data_review.loc[:int(0.2*len(huawei_data_review["Reviews"]))]
huawei_data_review_test = huawei_data_review.loc[int(0.2*len(huawei_data_review["Reviews"])):]

In [436]:
# get label
samsung_polarity_train = samsung_data_review_train["polarity"].tolist()
samsung_polarity_test = samsung_data_review_test["polarity"].tolist()
apple_polarity_train = apple_data_review_train["polarity"].tolist()
apple_polarity_test = apple_data_review_test["polarity"].tolist()
huawei_polarity_train = huawei_data_review_train["polarity"].tolist()
huawei_polarity_test = huawei_data_review_test["polarity"].tolist()

In [437]:
# Normalization
samsung_train_norm = normalize_corpus(np.array(samsung_data_review_train["Reviews"]))
samsung_test_norm = normalize_corpus(np.array(samsung_data_review_test["Reviews"]))

In [438]:
# Normalization
apple_train_norm = normalize_corpus(np.array(apple_data_review_train["Reviews"]))
apple_test_norm = normalize_corpus(np.array(apple_data_review_test["Reviews"]))

In [439]:
# Normalization
huawei_train_norm = normalize_corpus(np.array(huawei_data_review_train["Reviews"]))
huawei_test_norm = normalize_corpus(np.array(huawei_data_review_test["Reviews"]))

In [440]:
# TF IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range = (1,2)) 
samsung_matrix_TRAIN = vectorizer.fit_transform(samsung_train_norm).astype(float)
samsung_matrix_TEST = vectorizer.transform(samsung_test_norm)
#apple_matrix_TRAIN = vectorizer.fit_transform(apple_train_norm).astype(float)
#huawei_matrix_TRAIN = vectorizer.fit_transform(huawei_train_norm).astype(float)

In [441]:
# TF IDF Vectorization
apple_matrix_TRAIN = vectorizer.fit_transform(apple_train_norm).astype(float)
apple_matrix_TEST = vectorizer.transform(apple_test_norm)

In [442]:
# TF IDF Vectorization
huawei_matrix_TRAIN = vectorizer.fit_transform(huawei_train_norm).astype(float)
huawei_matrix_TEST = vectorizer.transform(huawei_test_norm)

In [443]:
# SVM based sentiment analysis Sumsung
from sklearn import linear_model

svm = linear_model.SGDClassifier(loss='hinge', random_state = 0) 
svm.fit(samsung_matrix_TRAIN, samsung_polarity_train)
predicted_svm_samsung = svm.predict(samsung_matrix_TEST) 
print('Sumsung Accuracy rate:', np.round(metrics.accuracy_score(samsung_polarity_test, predicted_svm_samsung), 3))

Sumsung Accuracy rate: 0.924


In [444]:
# SVM based sentiment analysis Apple
svm = linear_model.SGDClassifier(loss='hinge', random_state = 0) 
svm.fit(apple_matrix_TRAIN, apple_polarity_train)
predicted_svm_apple = svm.predict(apple_matrix_TEST) 
print('Apple Accuracy rate:', np.round(metrics.accuracy_score(apple_polarity_test, predicted_svm_apple), 3))

Apple Accuracy rate: 0.934


In [445]:
# SVM based sentiment analysis Huawei
svm = linear_model.SGDClassifier(loss='hinge', random_state = 0) 
svm.fit(huawei_matrix_TRAIN, huawei_polarity_train)
predicted_svm_huawei = svm.predict(huawei_matrix_TEST) 
print('Huawei Accuracy rate:', np.round(metrics.accuracy_score(huawei_polarity_test, predicted_svm_huawei), 3))

Huawei Accuracy rate: 0.882


In [446]:
'''
=============================================
SVM based sentiment analysis 
=============================================
'''
# Split train and test set
low_price_review_train = low_price_review[:int(0.2*len(low_price_review))]
low_price_review_test = low_price_review[int(0.2*len(low_price_review)):]
mid_price_review_train = mid_price_review[:int(0.2*len(mid_price_review))]
mid_price_review_test = mid_price_review[int(0.2*len(mid_price_review)):]
high_price_review_train = high_price_review[:int(0.2*len(high_price_review))]
high_price_review_test = high_price_review[int(0.2*len(high_price_review)):]

low_price_polarity_train = low_price_polarity[:int(0.2*len(low_price_polarity))]
low_price_polarity_test = low_price_polarity[int(0.2*len(low_price_polarity)):]
mid_price_polarity_train = mid_price_polarity[:int(0.2*len(mid_price_polarity))]
mid_price_polarity_test = mid_price_polarity[int(0.2*len(mid_price_polarity)):]
high_price_polarity_train = high_price_polarity[:int(0.2*len(high_price_polarity))]
high_price_polarity_test = high_price_polarity[int(0.2*len(high_price_polarity)):]

In [447]:
# Normalize & Vectorize
high_train_norm = normalize_corpus(np.array(high_price_review_train))
high_test_norm = normalize_corpus(np.array(high_price_review_test))
mid_train_norm = normalize_corpus(np.array(mid_price_review_train))
mid_test_norm = normalize_corpus(np.array(mid_price_review_test))
low_train_norm = normalize_corpus(np.array(low_price_review_train))
low_test_norm = normalize_corpus(np.array(low_price_review_test))

high_matrix_TRAIN = vectorizer.fit_transform(high_train_norm).astype(float)
high_matrix_TEST = vectorizer.transform(high_test_norm)
mid_matrix_TRAIN = vectorizer.fit_transform(mid_train_norm).astype(float)
mid_matrix_TEST = vectorizer.transform(mid_test_norm)
low_matrix_TRAIN = vectorizer.fit_transform(low_train_norm).astype(float)
low_matrix_TEST = vectorizer.transform(low_test_norm)

In [448]:
# SVM based sentiment analysis low price range
from sklearn import linear_model

svm = linear_model.SGDClassifier(loss='hinge', random_state = 0) 
svm.fit(low_matrix_TRAIN, low_price_polarity_train)
predicted_svm_low = svm.predict(low_matrix_TEST) 
print('Low price range Accuracy rate:', np.round(metrics.accuracy_score(predicted_svm_low,low_price_polarity_test), 3))

Low price range Accuracy rate: 0.903


In [450]:
# SVM based sentiment analysis mid price range
from sklearn import linear_model

svm = linear_model.SGDClassifier(loss='hinge', random_state = 0) 
svm.fit(mid_matrix_TRAIN, mid_price_polarity_train)
predicted_svm_mid = svm.predict(mid_matrix_TEST) 
print('Mid price range Accuracy rate:', np.round(metrics.accuracy_score(mid_price_polarity_test, predicted_svm_mid), 3))

Mid price range Accuracy rate: 0.929


In [451]:
pd.crosstab(pd.Series(predicted_svm_mid),
            pd.Series(mid_price_polarity_test),
            rownames = ['True:'], 
            colnames = ['Predicted:'], 
            margins = True)

Predicted:,negative,positive,All
True:,,,
negative,5,18,23
positive,16,48,64
All,21,66,87


In [452]:
# SVM based sentiment analysis high price range
from sklearn import linear_model

svm = linear_model.SGDClassifier(loss='hinge', random_state = 0) 
svm.fit(high_matrix_TRAIN, high_price_polarity_train)
predicted_svm_high = svm.predict(high_matrix_TEST) 
print('High price range Accuracy rate:', np.round(metrics.accuracy_score(high_price_polarity_test, predicted_svm_high), 3))

High price range Accuracy rate: 0.901


In [454]:
'''
=======================================
Neural network based sentiment analysis
=======================================
'''

'\n=======================================\nNeural network based sentiment analysis\n=======================================\n'

In [455]:
# Neural network based sentiment analysis Sumsung
from sklearn import neural_network

nn = neural_network.MLPClassifier() 
nn.fit(samsung_matrix_TRAIN, samsung_polarity_train)
predicted_nn_samsung = nn.predict(samsung_matrix_TEST)
print('Sumsung Accuracy rate:', np.round(metrics.accuracy_score(samsung_polarity_test, predicted_nn_samsung), 3))

Sumsung Accuracy rate: 0.92


In [456]:
# Neural network based sentiment analysis Apple
from sklearn import neural_network

nn = neural_network.MLPClassifier() 
nn.fit(apple_matrix_TRAIN, apple_polarity_train)
predicted_nn_apple = nn.predict(apple_matrix_TEST)
print('Apple Accuracy rate:', np.round(metrics.accuracy_score(apple_polarity_test, predicted_nn_apple), 3))

Apple Accuracy rate: 0.922


In [457]:
# Neural network based sentiment analysis Sumsung
from sklearn import neural_network

nn = neural_network.MLPClassifier() 
nn.fit(huawei_matrix_TRAIN, huawei_polarity_train)
predicted_nn_huawei = nn.predict(huawei_matrix_TEST)
print('Huawei Accuracy rate:', np.round(metrics.accuracy_score(huawei_polarity_test, predicted_nn_huawei), 3))

Huawei Accuracy rate: 0.88


In [458]:
# Neural network based sentiment analysis low price range
from sklearn import neural_network

nn = neural_network.MLPClassifier() 
nn.fit(low_matrix_TRAIN, low_price_polarity_train)
predicted_nn_low = nn.predict(low_matrix_TEST)
print('Low price range Accuracy rate:', np.round(metrics.accuracy_score(low_price_polarity_test, predicted_nn_low), 3))

Low price range Accuracy rate: 0.897


In [459]:
# Neural network based sentiment analysis mid price range
from sklearn import neural_network

nn = neural_network.MLPClassifier() 
nn.fit(mid_matrix_TRAIN, mid_price_polarity_train)
predicted_nn_mid = nn.predict(mid_matrix_TEST)
print('Mid price range Accuracy rate:', np.round(metrics.accuracy_score(mid_price_polarity_test, predicted_nn_mid), 3))

Mid price range Accuracy rate: 0.927


In [460]:
# Neural network based sentiment analysis high price range
from sklearn import neural_network

nn = neural_network.MLPClassifier() 
nn.fit(high_matrix_TRAIN, high_price_polarity_train)
predicted_nn_high = nn.predict(high_matrix_TEST)
print('High price range Accuracy rate:', np.round(metrics.accuracy_score(high_price_polarity_test, predicted_nn_high), 3))

High price range Accuracy rate: 0.91
